In [1]:
import pandas as pd

In [2]:
import scipy as sp
import scipy.linalg as spla
import scipy.optimize as spopt
import scipy.stats as spst
#sp?

In [3]:
import numpy as np
import numpy.linalg as npla

In [13]:
import statsmodels.api as sm

In [14]:
dat = sm.datasets.get_rdataset("Guerry", "HistData").data

In [16]:
smfOLS = sm.regression.linear_model.OLS.from_formula

model = 'Lottery ~ Literacy + Donations + Infants + Wealth + Commerce'
fit = smfOLS(model, data = dat).fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Lottery   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     8.392
Date:                Sun, 26 Sep 2021   Prob (F-statistic):           2.04e-06
Time:                        18:48:08   Log-Likelihood:                -380.11
No. Observations:                  86   AIC:                             772.2
Df Residuals:                      80   BIC:                             787.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     17.5098     11.827      1.480      0.143      -6.027      41.047
Literacy      -0.1499      0.166     -0.906      0.368      -0.479       0.180
Donations      0.0001      0.000      0.342      0.734      -0.001       0.001
Infants        0.0006      0.000      1.973      0.052   -5.08e-06       0.001
Wealth         0.3065      0.106      2.900      0.005       0.096       0.517
Commerce       0.1522      0.127      1.197      0.235      -0.101       0.405
==============================================================================
Omnibus:                        9.177   Durbin-Watson:                   1.874
Prob(Omnibus):                  0.010   Jarque-Bera (JB):                3.538
Skew:                          -0.165   Prob(JB):                        0.171
Kurtosis:                       2.062   Cond. No.                     1.17e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.17e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
from sklearn.linear_model import LinearRegression as LR

## Home assignment (graded, individual, in 2 week)
## email to pandreyanov@gmail.com, use ipnb, use markdown, explain what you do

### Assignment 1 (2 points).
Write a function assignment_1 that replicates the outcome of assignment_1_true on the *dat* dataset from the beginning of the notebook

In [32]:
def assignment_1_true(formula, data):
    fit = smfOLS(formula, data = data).fit()
    return fit.conf_int()

In [33]:
assignment_1_true('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat)

,0,1
Intercept,-6.027115,41.046645
Literacy,-0.479342,0.179528
Donations,-0.000675,0.000954
Infants,-0.000005,0.001164
Wealth,0.096173,0.516922
Commerce,-0.100867,0.405366


In [34]:
def assignment_1(formula, data):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    # add bias
    X = sm.add_constant(data[xcolumns].values)
    
    model = LR(fit_intercept=False).fit(X, Y)
    # model residuals
    eps = Y - model.predict(X)
    # pred std and coefs cov matrix
    sigma = eps.dot(eps) / (X.shape[0] - len(model.coef_))
    cov = sigma * npla.inv(X.T @ X)

    q_high = model.coef_ + 1.96*np.sqrt(np.diag(cov))
    q_low = model.coef_ - 1.96*np.sqrt(np.diag(cov))

    return pd.DataFrame({
        0 : q_low,
        1 : q_high
    }, index=['Intercept'] + xcolumns)

Make the cov matrix and process the std so as to make the confidence interval 


To find the confidence interval we further use formula with t-statistic 

In [35]:
assignment_1('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat)

,0,1
Intercept,-5.671549,40.691079
Literacy,-0.474365,0.174551
Donations,-0.000662,0.000942
Infants,0.000004,0.001155
Wealth,0.099351,0.513744
Commerce,-0.097043,0.401542


### Assignment 2 (3 points).
- Write a function that finds the coefficients for the **elastic net** regularized ols with coefficients $\lambda$ and $\mu$ of your choice
- This time you do not need to find the standard errors

In [36]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
import warnings
warnings.filterwarnings('ignore')

#### sklearn ElasticNet:

loss function:
$$
1 / (2 * n\_samples) * ||y - Xw||^2_2
+ alpha * l1\_ratio * ||w||_1
+ 0.5 * alpha * (1 - l1\_ratio) * ||w||^2_2
$$

control l1 and l2 separately:
$$
\lambda * ||w||_1 + \mu * ||w||_2^2
$$

$$
\alpha = \lambda + 2\mu
$$
$$
l1\_ratio = \lambda / (\lambda + 2\mu)
$$

In [37]:
def get_sklearn_elastic_params(lamda, mu):
    alpha = lamda + 2*mu
    l1_ratio = lamda / (lamda+2*mu)
    return alpha, l1_ratio

In [38]:
def assignment_2(formula, data, mu, lamda):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    if lamda == 0 and mu == 0:
        # no regularization
        model = LR(fit_intercept=False)
    else:
        alpha, l1_ratio = get_sklearn_elastic_params(lamda, mu)
        # your mock results
        model = ElasticNet(fit_intercept=False, alpha=alpha, l1_ratio=l1_ratio)
        
    model = model.fit(X, Y)
    
    df = pd.DataFrame(model.coef_, columns = [0])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [39]:
assignment_2('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat, 1, 1)

,0
Intercept,0.000000
Literacy,0.056924
Donations,0.000186
Infants,0.000791
Wealth,0.321767
Commerce,0.238890


Conclusion

- we see, that model has zero coef on Intercept, Donations, Ifants. So, these features aren't important for Lottery prediction

### Assignment 3 (5 points)
- Write a function that finds the coefficients for the **elastic net** regularized ols with crossvalidation
- Use number of folds of your choice
- You do not need to find the standard errors
- Search for $\lambda$, $\mu$ in the range [0,5]x[0,5]

In [40]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import itertools

target metric: MSE

In [56]:
def run_cv(model, X: np.array, Y: np.array, folds: int):
    cv = KFold(n_splits=folds)
    errs = []
    for train_idx, val_idx in cv.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx],Y[val_idx]
        
        model.fit(X_train, y_train)
        err = mean_squared_error(y_val, model.predict(X_val))
        errs.append(err)
        
    return np.mean(errs)

In [62]:
def assignment_3(formula, data, folds = 5):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    hyperparams = []
    mean_mses = []
    
    for lamda, mu in itertools.product(np.linspace(0.1, 5), np.linspace(0.1, 5)):
        hyperparams.append((lamda, mu))
        alpha, l1_ratio = get_sklearn_elastic_params(lamda, mu)
        model = ElasticNet(fit_intercept=False, alpha=alpha, l1_ratio=l1_ratio)
        mean_mses.append(run_cv(model, X, Y, folds))
        
    best_lambda, best_mu = hyperparams[np.argmin(mean_mses)]
    best_mse = min(mean_mses)
    
    df = assignment_2(formula, data, mu=best_mu, lamda=best_lambda)
    df_info = pd.DataFrame({
        'best_lambda' : best_lambda,
        'best_mu' : best_mu,
        'best_mse' : best_mse
    }, index=[0])
    
    return df, df_info

In [66]:
res, info = assignment_3('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat, folds = 4)

In [67]:
res

,0
Intercept,0.000000
Literacy,0.051575
Donations,0.000207
Infants,0.000821
Wealth,0.316535
Commerce,0.230403


In [68]:
info

,best_lambda,best_mu,best_mse
0,5.0,5.0,530.012275


Conclusion:
- we see, that best score achieved on strongest regularization

### Assignment 4 (5 points).
- Write a function that finds the coefficients for the **ridge** regularized ols with coefficient $\lambda$ crossvalidated
- This time you HAVE TO to find the standard errors
- Derive the standard errors in markdown, assuming homoskedasticity of errors $E[ee'|X]=\sigma^2 I$
- see, for example, https://lukesonnet.com/teaching/inference/200d_standard_errors.pdf

$$
\hat{\sigma}^2 = \overline{diag(ee')}
$$
$$
\quad \hat{Cov}(\beta) = \hat{\sigma}^2 (X'X + \lambda I)^{-1} X'X [(X'X + \lambda I)^{-1}]'
$$

In [57]:
from sklearn.linear_model import Ridge

In [87]:
def assignment_4(formula, data, folds = 5):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    hyperparams = []
    mean_mses = []
    
    # your mock results
    for lamda in np.linspace(0.1, 5):
        hyperparams.append(lamda)
        model = Ridge(fit_intercept=False, alpha=lamda)
        mean_mses.append(run_cv(model, X, Y, folds))
        
    best_lambda = hyperparams[np.argmin(mean_mses)]
    best_mse = min(mean_mses)
    
    model = Ridge(fit_intercept=False, alpha=best_lambda)
    model.fit(X, Y)
    
    resids = Y - model.predict(X)
    sigma_sq = np.mean(np.diag(resids.reshape(-1, 1) @ resids.reshape(1, -1)))
    
    first_comp = npla.inv(X.T@X + best_lambda*np.eye(X.shape[1]))
    cov = sigma_sq * (first_comp @ (X.T@X) @ first_comp.T)
    coef_stds = np.sqrt(np.diag(cov))
    
    df = pd.DataFrame(model.coef_, columns = ['coef'])
    df.index = np.array(['Intercept'] + xcolumns)
    df['std'] = coef_stds
    
    info = pd.DataFrame({
        'best_lambda' : best_lambda,
        'best_mse' : best_mse
    }, index=[0])
    return df, info

In [88]:
res, info = assignment_4('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat, folds = 3)

In [89]:
res

,coef,std
Intercept,6.710674,4.393657
Literacy,-0.021500,0.099478
Donations,0.000165,0.000396
Infants,0.000705,0.000256
Wealth,0.316712,0.101982
Commerce,0.207011,0.110974


In [90]:
info

,best_lambda,best_mse
0,5.0,534.111282


Conclusion:

- we see, that best loss achieves with strongest regularization
- coefs with regularization have small std